In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [39]:
res = requests.get('https://www.mahkotamedical.com/find-a-doctor/')
soup = BeautifulSoup(res.text,'lxml')

In [40]:
docs = soup.select('.pt-cv-pagination')
# pages = docs[0].find('ul',{'class':'.pt-cv-pagination .pt-cv-ajax .pagination'})['data-totalpages']
pages = int(docs[0]['data-totalpages'])
pages

8

In [44]:
all_content = BeautifulSoup()
for i in range(1,9):
    new_res = requests.get('https://www.mahkotamedical.com/find-a-doctor/?_page='+str(i))
    all_content.append(BeautifulSoup(new_res.text,'lxml'))

In [69]:
fil = all_content.select('.pt-cv-href-thumbnail')
links_to_doc = [i['href'] for i in fil] 

In [155]:
def get_result(links):
    name,spe,qual,email,avail,lang = [],[],[],[],[],[]
    for each in links:
        details = requests.get(each)
        content = BeautifulSoup(details.text,'lxml')
        name.append(content.select('.doctor_title')[0].text)
        spe.append(content.select('.speciality_field')[0].text)
        education = content.select('.field_5c0656ee71864')[0].findAll('p')
        edu = [i.text for i in education]
        qual.append(edu)
        time = content.select('.field_5c06565e7185d')[0].text.split('\n\r\n')
        time = [i.replace('\r\n',' ') for i in time]
        avail.append(time)
        lang.append(content.select('.field_5c32b9ee273d5')[0].text.split(','))
    df = pd.DataFrame({
        'Name':name,
        'Specialization':spe,
        'Qualification':qual,
        'Availability':avail,
        'Languages':lang
    })
    return df

In [150]:
one = links_to_doc[0]
details = requests.get(one)
content = BeautifulSoup(details.text,'lxml')
doc_name = content.select('.doctor_title')[0].text
speciality = content.select('.speciality_field')[0].text
education = content.select('.field_5c0656ee71864')[0].findAll('p')
edu = [i.text for i in education]
mail = content.select('a')
time = content.select('.field_5c06565e7185d')[0].text.split('\n\r\n')
time = [i.replace('\r\n',' ') for i in time]
languages = content.select('.field_5c32b9ee273d5')[0].text.split(',')
languages

['English', ' Bahasa Melayu', ' Bahasa Indonesia']

In [157]:
df = get_result(links_to_doc)
df.to_json('Doctor_Details.json')

In [158]:
df.to_json('Details.json',orient='index')